<a href="https://colab.research.google.com/github/sakuna47/Stock_Price_Prediction-/blob/Backend/Stock_Price_Prediction_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Enhanced Stock Price Prediction Model
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:

# =============================================================================
# CONFIGURATION
# =============================================================================
TICKER = "AAPL"
START_DATE = "2015-01-01"
END_DATE = "2025-06-25"
PREDICTION_DAYS_AHEAD = 7  #  (1=next day, 5=next week, etc.)
# =============================================================================